# Demand Forecasting for Retail\nBeginner-friendly notebook: generate sample data, do simple EDA, and example forecast cells (Prophet / ARIMA / LSTM).\n\nIf a package fails to install, run the simpler cells (data load, baseline) first.

In [ ]:
# 1) Imports and helper metrics\nimport pandas as pd\nimport numpy as np\nfrom matplotlib import pyplot as plt\nimport math\nfrom sklearn.metrics import mean_squared_error\n\ndef rmse(y_true, y_pred):\n    return math.sqrt(mean_squared_error(y_true, y_pred))\n\ndef mape(y_true, y_pred):\n    y_true, y_pred = np.array(y_true), np.array(y_pred)\n    mask = y_true != 0\n    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100\n\nprint('helpers ready')

In [ ]:
# 2) If you haven't generated data yet, run the generator (from project root):\n# python data/data_generator.py\n\ntry:\n    df = pd.read_csv('../data/retail_sales.csv', parse_dates=['date'])\nexcept Exception as e:\n    try:\n        df = pd.read_csv('data/retail_sales.csv', parse_dates=['date'])\n    except Exception as e2:\n        print('data/retail_sales.csv not found. Run data/data_generator.py first.')\n        raise\n\nprint('Loaded data, shape =', df.shape)\ndf.head()

In [ ]:
# 3) Select a single time series (example: store_1 - prod_A) and do a quick plot\nseries = df[(df.store=='store_1') & (df.product=='prod_A')].copy()\nseries = series.sort_values('date').set_index('date').resample('D').sum()['sales'].fillna(0)\nseries.plot(figsize=(12,4), title='Daily sales: store_1 - prod_A')\nplt.show()

### Prophet example (optional)\nInstall prophet first: `pip install prophet`\nIf installation fails, skip this cell and try ARIMA or the baseline LSTM example below.

In [ ]:
try:\n    from prophet import Prophet\n    train = series[:'2020-12-31']\n    test = series['2021-01-01':]\n    prophet_df = train.reset_index().rename(columns={'date':'ds','sales':'y'})\n    m = Prophet()\n    m.fit(prophet_df)\n    future = m.make_future_dataframe(periods=len(test), freq='D')\n    fcst = m.predict(future)\n    preds = fcst.set_index('ds')['yhat'][-len(test):]\n    print('Prophet RMSE:', rmse(test.values, preds.values), 'MAPE:', mape(test.values, preds.values))\nexcept Exception as e:\n    print('Prophet cell failed or prophet not installed:', e)

### ARIMA example (uses pmdarima auto_arima)
Install: `pip install pmdarima`

In [ ]:
try:\n    from pmdarima import auto_arima\n    train = series[:'2020-12-31']\n    test = series['2021-01-01':]\n    ar = auto_arima(train, seasonal=True, m=7, stepwise=True, suppress_warnings=True)\n    ar_pred = ar.predict(n_periods=len(test))\n    print('ARIMA RMSE:', rmse(test.values, ar_pred), 'MAPE:', mape(test.values, ar_pred))\nexcept Exception as e:\n    print('ARIMA cell failed or pmdarima not installed:', e)

### Simple LSTM example (optional)\nInstall tensorflow: `pip install tensorflow` (this may be slow/heavy).

In [ ]:
try:\n    from sklearn.preprocessing import MinMaxScaler\n    from tensorflow.keras.models import Sequential\n    from tensorflow.keras.layers import LSTM, Dense\n    train = series[:'2020-12-31']\n    test = series['2021-01-01':]\n    scaler = MinMaxScaler()\n    train_scaled = scaler.fit_transform(train.values.reshape(-1,1))\n    SEQ = 30\n    def create_seq(data, seq_len=30):\n        X, y = [], []\n        for i in range(len(data)-seq_len):\n            X.append(data[i:i+seq_len])\n            y.append(data[i+seq_len])\n        return np.array(X), np.array(y)\n    X_train, y_train = create_seq(train_scaled, SEQ)\n    model_l = Sequential()\n    model_l.add(LSTM(64, input_shape=(SEQ,1)))\n    model_l.add(Dense(1))\n    model_l.compile(optimizer='adam', loss='mse')\n    model_l.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)\n    # forecasting (simple loop)\n    history = train_scaled[-SEQ:].tolist()\n    preds = []\n    for _ in range(len(test)):\n        x = np.array(history[-SEQ:]).reshape(1, SEQ, 1)\n        p = model_l.predict(x)[0,0]\n        preds.append(p)\n        history.append([p])\n    preds_inv = scaler.inverse_transform(np.array(preds).reshape(-1,1)).ravel()\n    print('LSTM RMSE:', rmse(test.values, preds_inv), 'MAPE:', mape(test.values, preds_inv))\nexcept Exception as e:\n    print('LSTM cell failed or tensorflow not installed:', e)

## Save results (example)
This notebook saves predictions to `results_forecasts.csv` (if cells run successfully).